# División del dataset que contiene audio features para usar en DeepFM

In [ ]:
!pip install pyreclab --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install psutil

In [ ]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 25.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pyreclab
import numpy as np
import scipy.sparse as sparse
import implicit
import psutil
import time
import random
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Cargamos los datos y vemos las primeras 5 filas

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/RecSys/data/playlistid_itemid_1000p_train.txt'
test_path = '/content/drive/MyDrive/Colab Notebooks/RecSys/data/test_challenge_200p.txt'
metadata_path = '/content/drive/MyDrive/Colab Notebooks/RecSys/data/id2metadata.csv'
train_set = pd.read_csv(train_path, delim_whitespace=True, header=None, names=['playlist_id', 'track_id'])
test_set = pd.read_csv(test_path, delim_whitespace=True, header=None, names=['playlist_id', 'track_id'])
metadata_file = pd.read_csv(metadata_path)
metadata_file.rename(columns={'id': 'track_id'}, inplace=True)



<ipython-input-8-93dc4e10e164>:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  train_set = pd.read_csv(train_path, delim_whitespace=True, header=None, names=['playlist_id', 'track_id'])
<ipython-input-8-93dc4e10e164>:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  test_set = pd.read_csv(test_path, delim_whitespace=True, header=None, names=['playlist_id', 'track_id'])


In [ ]:
train_set.head(5)

,playlist_id,track_id
0,0,9700
1,0,22032
2,0,8608
3,0,5364
4,0,7630


In [ ]:
test_set.head(5)

,playlist_id,track_id
0,2,19630
1,2,9539
2,2,17752
3,2,29381
4,2,31252


In [ ]:
metadata_file.head(5)

,danceability,energy,key,valence,tempo,track_id
0,0.303,0.616,10,0.219,78.673,20966
1,0.748,0.627,7,0.524,120.963,19094
2,0.491,0.324,2,0.123,115.137,149
3,0.861,0.417,0,0.235,119.968,2787
4,0.750,0.808,10,0.263,140.158,17215


In [ ]:
metadata_file.describe()

,danceability,energy,key,valence,tempo,track_id
count,34440.000000,34440.000000,34440.000000,34440.000000,34440.000000,34440.000000
mean,0.584677,0.636589,5.213618,0.486122,121.728991,17220.902671
std,0.164397,0.225415,3.583492,0.243592,29.304274,9942.696508
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.476000,0.490000,2.000000,0.292000,98.656750,8610.750000
50%,0.592000,0.669000,5.000000,0.477000,120.926000,17220.500000
75%,0.705000,0.818000,8.000000,0.677000,140.033250,25830.250000
max,0.988000,1.000000,11.000000,0.998000,219.297000,34443.000000


Al analizar los metadatos, vemos que sería adecuado normalizar el tempo que está en una escala distinta al resto de valores continuos. No es necesario hacer esto con key ya que es un valor categórico y DeepFM lo recibe por separado

In [ ]:
# Calcular los valores mínimo y máximo de "tempo" a partir de metadata_file
tempo_min = metadata_file['tempo'].min()
tempo_max = metadata_file['tempo'].max()

# Agregar la columna 'tempo_normalized' en metadata_file
metadata_file['tempo'] = (metadata_file['tempo'] - tempo_min) / (tempo_max - tempo_min)


Para usar el dataset en DeepFM sin problemas, reemplazaremos los posibles -1 que podría tener "key" con 12. Aunque sabemos que no existen filas que tengan key = -1, es mejor dejarlo definido por si esto cambia en un futuro

In [ ]:
metadata_file['key'] = metadata_file['key'].replace(-1, 12)


In [ ]:
metadata_file.describe()

,danceability,energy,key,valence,tempo,track_id
count,34440.000000,34440.000000,34440.000000,34440.000000,34440.000000,34440.000000
mean,0.584677,0.636589,5.213618,0.486122,0.555087,17220.902671
std,0.164397,0.225415,3.583492,0.243592,0.133628,9942.696508
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.476000,0.490000,2.000000,0.292000,0.449877,8610.750000
50%,0.592000,0.669000,5.000000,0.477000,0.551426,17220.500000
75%,0.705000,0.818000,8.000000,0.677000,0.638555,25830.250000
max,0.988000,1.000000,11.000000,0.998000,1.000000,34443.000000


Vemos que tempo quedó efectivamente entre 0 y 1 como el resto de metadatos continuos.

In [ ]:
train_set = train_set.merge(metadata_file, on='track_id', how='left')
test_set = test_set.merge(metadata_file, on='track_id', how='left')

# Estadísticas descriptivas

In [ ]:
train_set.describe()

,playlist_id,track_id,danceability,energy,key,valence,tempo
count,51822.000000,51822.000000,51819.000000,51819.000000,51819.000000,51819.000000,51819.000000
mean,517.060515,17236.197484,0.610660,0.642072,5.171250,0.501458,0.554642
std,285.541561,9956.884789,0.161181,0.208900,3.612226,0.238396,0.132073
min,0.000000,1.000000,0.000000,0.000020,0.000000,0.000000,0.000000
25%,269.000000,8732.250000,0.504000,0.508000,2.000000,0.314000,0.451488
50%,518.000000,17279.000000,0.620000,0.671000,5.000000,0.496000,0.548735
75%,767.000000,25841.000000,0.728000,0.806000,8.000000,0.690000,0.638502
max,999.000000,34442.000000,0.988000,1.000000,11.000000,0.998000,0.998495


In [ ]:
test_set.describe()

,playlist_id,track_id,danceability,energy,key,valence,tempo
count,15681.00000,15681.000000,15681.000000,15681.000000,15681.000000,15681.000000,15681.000000
mean,479.65557,17027.657420,0.602461,0.650499,5.163319,0.479442,0.555722
std,286.88573,9937.256282,0.157728,0.205672,3.617143,0.235172,0.130181
min,2.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,234.00000,8474.000000,0.499000,0.518000,2.000000,0.294000,0.455711
50%,453.00000,16958.000000,0.609000,0.676000,5.000000,0.465000,0.548758
75%,691.00000,25588.000000,0.715000,0.812000,8.000000,0.659000,0.638559
max,997.00000,34443.000000,0.979000,0.999000,11.000000,0.991000,1.000000


En el conjunto de entrenamiento hay 3 filas faltantes, por lo que tendremos que identificarlas y eliminarlas. En el conjunto de test notamos que no hay filas faltantes, todas las columnas tienen el mismo numero. También notamos que el valor mínimo de key es 0 y considerando que -1 indica que no se detectó la nota, esto significa que se identificó el key en todas las canciones.

In [ ]:
missing_metadata = train_set[train_set.isnull().any(axis=1)]
missing_metadata


,playlist_id,track_id,danceability,energy,key,valence,tempo
10512,223,26556,NaN,NaN,NaN,NaN,NaN
20938,438,31379,NaN,NaN,NaN,NaN,NaN
38675,762,31523,NaN,NaN,NaN,NaN,NaN


Eliminamos estas columnas

In [ ]:
train_set = train_set.dropna()

Para poder usar el dataset en DeepFM, neecesitamos agregar una columna target que será 1 para todas las filas. Luego agregaremos canciones aleatorias que no están presentes en la playlist y tendrán target = 0

Negative Sampling para conjunto de train

In [ ]:
positive_train = train_set.copy()
positive_train['target'] = 1

# Crear los datos negativos
negative_train = []
for playlist_id in train_set['playlist_id'].unique():
    all_songs = set(metadata_file['track_id'].unique())
    playlist_songs = set(train_set[train_set['playlist_id'] == playlist_id]['track_id'])
    negative_songs = list(all_songs - playlist_songs)

    # Controlar el número de negativos a generar (igual a positivos por playlist)
    num_negatives = len(playlist_songs)
    if len(negative_songs) < num_negatives:
        num_negatives = len(negative_songs)

    for song_id in random.sample(negative_songs, num_negatives):
        # Agregar playlist_id y track_id
        negative_sample = {'playlist_id': playlist_id, 'track_id': song_id, 'target': 0}

        # Agregar los metadatos de la canción
        metadata = metadata_file[metadata_file['track_id'] == song_id].iloc[0].to_dict()
        negative_sample.update(metadata)

        negative_train.append(negative_sample)

negative_train = pd.DataFrame(negative_train)

# Combinar positivos y negativos
train_set_final = pd.concat([positive_train, negative_train], ignore_index=True)
train_set_final = train_set_final.sample(frac=1, random_state=42).reset_index(drop=True)



In [ ]:
train_set_final.head(5)

,playlist_id,track_id,danceability,energy,key,valence,tempo,target
0,762,22104.0,0.492,0.870,0.0,0.4090,0.730261,0
1,115,95.0,0.724,0.508,11.0,0.0629,0.556597,0
2,256,32462.0,0.707,0.750,10.0,0.7910,0.364766,0
3,877,18100.0,0.662,0.875,5.0,0.1320,0.583688,1
4,689,16276.0,0.438,0.872,0.0,0.4780,0.738227,0


In [ ]:
# Crear los datos positivos
positive_test = test_set.copy()
positive_test['target'] = 1

# Crear los datos negativos
negative_test = []
for playlist_id in test_set['playlist_id'].unique():
    all_songs = set(metadata_file['track_id'].unique())
    playlist_songs = set(test_set[test_set['playlist_id'] == playlist_id]['track_id'])
    negative_songs = list(all_songs - playlist_songs)

    # Generar más negativos por playlist (por ejemplo, 99 negativos por positivo)
    num_negatives = len(playlist_songs) * 4
    if len(negative_songs) < num_negatives:
        num_negatives = len(negative_songs)

    for song_id in random.sample(negative_songs, num_negatives):
        # Agregar playlist_id y track_id
        negative_sample = {'playlist_id': playlist_id, 'track_id': song_id, 'target': 0}

        # Agregar los metadatos de la canción
        metadata = metadata_file[metadata_file['track_id'] == song_id].iloc[0].to_dict()
        negative_sample.update(metadata)

        negative_test.append(negative_sample)

negative_test = pd.DataFrame(negative_test)

# Combinar positivos y negativos
test_set_final = pd.concat([positive_test, negative_test], ignore_index=True)
test_set_final = test_set_final.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
test_set_final.head(5)

,playlist_id,track_id,danceability,energy,key,valence,tempo,target
0,584,4438.0,0.628,0.686,7.0,0.474,0.385076,0
1,847,5572.0,0.462,0.767,10.0,0.901,0.916857,1
2,840,9353.0,0.475,0.848,5.0,0.520,0.405696,0
3,863,1681.0,0.509,0.802,0.0,0.212,0.638896,0
4,600,17568.0,0.648,0.736,8.0,0.378,0.469806,1


Finalmente, guardamos los conjuntos generados.

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/RecSys/data/training_set.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/RecSys/data/test_set.csv'
item_path = '/content/drive/MyDrive/Colab Notebooks/RecSys/data/item_metadata.csv'

In [ ]:
train_shuffled = train_set_final.sample(frac=1, random_state=42).reset_index(drop=True)
test_shuffled = test_set_final.sample(frac=1, random_state=42).reset_index(drop=True)

train_shuffled.to_csv(train_path, index=False)
test_shuffled.to_csv(test_path, index=False)

In [ ]:
train_set_final.to_csv(train_path, index=False)
test_set_final.to_csv(test_path, index=False)